In [12]:
%pylab inline

import pickle
import scipy.stats as stats
import scipy.optimize as opt
import deribit_api as api

rcParams['axes.prop_cycle'] = cycler('color',pickle.load(open('tableau.colors', 'rb')))

Populating the interactive namespace from numpy and matplotlib


In [19]:
def BSsetup(S, K, T, sigma, r, c):
    d1 = (np.log(S/K) + (r - c + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return d1,d2

def BScall(S, K, T, sigma, r, c):
    d1, d2 = BSsetup(S, K, T, sigma, r, c)
    return stats.norm.cdf(d1)*S - stats.norm.cdf(d2)*K*np.exp(-r*T)

def BSput(S, K, T, sigma, r, c):
    d1, d2 = BSsetup(S, K, T, sigma, r, c)
    return stats.norm.cdf(-d2)*K*np.exp(-r*T) - stats.norm.cdf(-d1)*S

def BSvol(S, K, T, r, c, V, call=True):
    func = lambda sigma: BScall(S,K,T,sigma,r,c)-V if call else BSput(S,K,T,sigma,r,c)-V
    return opt.brentq(func, 0, 10.0)

In [4]:
def T(expiry):
    diff = datetime.datetime.strptime('%s 20:00:00'%expiry, '%Y-%m-%d %H:%M:%S') - datetime.datetime.today()
    return diff.total_seconds()/3600/24/360

In [8]:
keys = open('keys.txt', 'r').read().split('\n')[:-1]
client = api.RestClient(keys[0], keys[1])

In [10]:
instruments = client.getinstruments()

In [20]:
BScall(100.00, 100.00, 1.0, 1.0, 0.0, 0.05)

38.24850929237337

In [22]:
BSvol(100.00, 100.00, 1.0, 0.0, 0.05, 40)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


0.0